In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import jieba
import numpy as np
from sklearn.impute import SimpleImputer

# 1 特征中含有字符串的时候（当成类别），如何做特征提取

In [ ]:
def dictvec():
    """
    字典数据提取
    :return: None
    """
    # 实例化
    # sparse改为True，输出的是每个不为零位置的坐标，稀疏矩阵可以节省存储空间
    # 矩阵中存在大量的0，sparse存储只记录非零位置，节省空间的作用
    #Vectorizer中文含义是矢量器的含义
    dict1 = DictVectorizer(sparse=True)  # 把sparse设置为True，输出的是稀疏矩阵
    # 每个样本是一个字典，字典的key是特征名，value是特征值，有三个样本
    #调用fit_transform方法，把字典转换为稀疏矩阵
    data = dict1.fit_transform([{'city': '北京', 'temperature': 100},
                                {'city': '上海', 'temperature': 60},
                                {'city': '深圳', 'temperature': 30}])
    print(data)
    print('-' * 50)
    # 字典中一些类别数据，分别进行转换成特征
    print(dict1.get_feature_names_out())
    print("-" * 50)
    print(dict1.inverse_transform(data))  # 去看每个特征代表的含义，逆转回去

    return None

dictvec()